In [2]:
import numpy as np
import pandas as pd
import sys
import datetime
import psycopg
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 1. Read data

In [5]:
# Just use my own path?
path = "/Users/wangyicheng/Documents/GitHub/36614-Geese/Data/Quality/"

# info1 = pd.read_csv(path+sys.argv[2])
info1 = pd.read_csv(path+"Hospital_General_Information-2021-07.csv")

## 2. Preprocess data

In [6]:
# Replace 'Not Available' value to NaN
info1 = info1.replace('Not Available', np.nan)

In [48]:
## Insert date column as python date object
date = '2022-01-01'
# date = date.split('-')
# x = datetime.date(int(date[0]), int(date[1]), int(date[2]))
# date = x.strftime("%Y-%m-%d")
# type(date)

In [49]:
info1['Rating year'] = date
rate_table = info1.loc[:,["Facility ID","Hospital overall rating", "Rating year"]]

In [83]:
rate_table["Facility ID"] = rate_table["Facility ID"].astype('string')
rate_table["Hospital overall rating"] = rate_table["Hospital overall rating"].astype('Int64')
rate_table['Rating year'] = pd.to_datetime(rate_table['Rating year'], format="%Y-%m-%d")

In [84]:
rate_table.dtypes

Facility ID                        string
Hospital overall rating             Int64
Rating year                datetime64[ns]
dtype: object

## 3. Load data into psql

In [85]:
conn = psycopg.connect(
    host="sculptor.stat.cmu.edu", dbname="yicheng6",
    user="yicheng6", password="Oor4cah8p"
)
cur = conn.cursor()

In [86]:
# Container to record insert failed row
key = ["hospital_pk","Hospital overall rating", "Rating year"]
df_error = pd.DataFrame(columns=key)

In [88]:
num_rows_inserted = 0

# make a new transaction
with conn.transaction():
    
    for index, row in rate_table.iterrows():
        #try:
            # make a new SAVEPOINT -- like a save in a video game
            cur.execute("SAVEPOINT save1")
            with conn.transaction():  
                # now insert  (hospital_pk, rating_year, rating) into the data
                insert = ("INSERT INTO Rating_Time",
                          "VALUES (%s, %s, %s)")
                cur.execute(insert, tuple(row))
#         except Exception as e:
#             # if an exception/error happens in this block, Postgres goes back to
#             # the last savepoint upon exiting the `with` block
#             print("insert failed in row " + str(index))
#             df_error = pd.concat([df_error, row])

#             # add additional logging, error handling here
#         else:
#             # no exception happened, so we continue without reverting the savepoint
#             num_rows_inserted += 1
            
#     print(num_rows_inserted)
#     df_error.to_csv("Error_row.csv", index = False)

# # now we commit the entire transaction
# # conn.commit()


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=sculptor.stat.cmu.edu database=yicheng6) at 0x7fd4b964dd00>

TypeError: expected string or bytes-like object